TODO : Mise en page du notebook

## Importation

In [1]:
from azureml.core import Model
from azureml.core import Workspace, Environment
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

import os
import requests
import json
from dotenv import load_dotenv

load_dotenv()

True

source : https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python

Importation des clés

In [2]:
# clés
tenant_id = os.getenv("YOUR_ML_WORKSPACE_TENANT_ID")
subscription_id = os.getenv("YOUR_ML_WORKSPACE_SUBSCRIPTION_ID")
resource_group = os.getenv("YOUR_ML_WORKSPACE_RESOURCE_GROUP")
workspace_name = os.getenv("YOUR_ML_WORKSPACE_WORKSPACE_NAME")

Authentification

In [3]:
ia = InteractiveLoginAuthentication(tenant_id=tenant_id)

ws = Workspace.get(name=workspace_name,
                   subscription_id=subscription_id,
                   resource_group=resource_group,
                   auth=ia)

enregistrement du modèle

In [4]:
model = Model.register(
    workspace=ws,
    model_path='data/deploiement',
    model_name='lstm_glove',
    description='Modèle avancé - Analyse Sentiment - LSTM + GloVe')

Registering model lstm_glove


déploiment

In [5]:
myenv = Environment('modelavance')
myenv.python.conda_dependencies.add_pip_package("pickle5")
myenv.python.conda_dependencies.add_pip_package("tensorflow")
myenv.python.conda_dependencies.add_pip_package("azureml-core")

In [6]:
inference_config = InferenceConfig(environment=myenv,
                                   entry_script='score.py')

VM

In [7]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores=0.5,
                                                       memory_gb=1)

démarrage

In [8]:
service = Model.deploy(workspace=ws,
                       name='tweet-sentiment-service',
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config)


service.wait_for_deployment(show_output=True)

# Verbose
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-03-01 00:15:21+01:00 Creating Container Registry if not exists.
2022-03-01 00:15:21+01:00 Registering the environment.
2022-03-01 00:15:24+01:00 Use the existing image.
2022-03-01 00:15:24+01:00 Generating deployment configuration.
2022-03-01 00:15:26+01:00 Submitting deployment to compute.
2022-03-01 00:15:31+01:00 Checking the status of deployment tweet-sentiment-service..
2022-03-01 00:18:11+01:00 Checking the status of inference endpoint tweet-sentiment-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


logs

In [9]:
print(service.get_logs())

2022-02-28T23:17:49,488164200+00:00 - iot-server/run 
2022-02-28T23:17:49,467062000+00:00 - rsyslog/run 
2022-02-28T23:17:49,522216500+00:00 - nginx/run 
2022-02-28T23:17:49,539705500+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-02-28T23:17:50,126935600+00:00 - iot-server/finish 1 0
2022-02-28T23:17:50,128283800+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (66)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 90
2022-02-28 23:17:52.152200: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /azureml-envs/azureml_6d1e37acf0f20b727888d9dbb084e881/lib:/azureml-envs/azureml_6d1e37acf0f20b727888d9dbb084e

essai

In [10]:
rest_endpoint = 'YOUR_REST_ENDPOINT_HERE'
headers = {'Content-Type': 'application/json'}

# Provide a text example
data = json.dumps({'text': 'I told you that she was not happy'})

# Call with POST request
response = requests.post(rest_endpoint, data=data, headers=headers)

# Print result
print('Status : ', response.status_code)
print('Tweet : ', (json.loads(data)).get('text'))
print('Label : ', (response.json()).get('Label'))
print('Score : ', (response.json()).get('Score'))
print('Temps : ', (response.json()).get('Temps'))

Status :  200
Tweet :  I told you that she was not happy
Label :  Négatif
Score :  0.4459971487522125
Temps :  1.0060954093933105


arrêt des services

In [11]:
service.delete()
model.delete()